In [ ]:
import gym
from model import Network
import numpy as np
from sharedOptimizer import sharedAdam
import time
import torch
import torch.multiprocessing as mp
from train import train
from test import test
import os
os.environ["OMP_NUM_THREADS"] = "1"

class params:
    env_name = "BipedalWalker-v2"
    n_steps = 20
    gamma = 0.99
    tau = 0.98
    lr = 0.0001
    seed = 1
    processes_amount = 8
    max_episode_length = 10000
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    solved_score = 300

torch.manual_seed(params.seed)
env = gym.make(params.env_name)
print("observation = ", env.observation_space.shape[0])
print("action_space = ", env.action_space)
shared_model = Network(env.observation_space.shape[0], env.action_space, params.device).to(params.device)
shared_model.share_memory()
shared_optimizer = sharedAdam(shared_model.parameters(), lr = params.lr)
shared_optimizer.share_memory()

processes = []
p = mp.Process(target=test, args=(params,shared_model))
p.start()
processes.append(p)
time.sleep(0.2)

for i in range(params.processes_amount):
    p = mp.Process(target=train, args=(i, params, shared_model, shared_optimizer))
    p.start()
    processes.append(p)
    time.sleep(0.2)
for p in processes:
    time.sleep(0.2)
    p.join()

observation =  24
action_space =  Box(4,)
1 |reward: -92.73437759753224 |last: -92.73437759753224 |max_reward: -10000000 |ts: 129
   NEW MAX saving model   : -92.73437759753224
2 |reward: -101.39227188159401 |last: -97.06332473956313 |max_reward: -92.73437759753224 |ts: 46
3 |reward: -99.43122015604067 |last: -97.8526232117223 |max_reward: -92.73437759753224 |ts: 48
4 |reward: -91.90953784665648 |last: -96.36685187045585 |max_reward: -92.73437759753224 |ts: 80
   NEW MAX saving model   : -91.90953784665648
5 |reward: -97.4424677848077 |last: -96.58197505332622 |max_reward: -91.90953784665648 |ts: 60
6 |reward: -87.22111433220468 |last: -95.02183159980596 |max_reward: -91.90953784665648 |ts: 139
   NEW MAX saving model   : -87.22111433220468
7 |reward: -84.5259438574978 |last: -93.5224190651905 |max_reward: -87.22111433220468 |ts: 175
   NEW MAX saving model   : -84.5259438574978
8 |reward: 108.64920514514094 |last: -68.25096603889908 |max_reward: -84.5259438574978 |ts: 1399
   NEW MAX 